In [1]:
import sys
import cv2
import numpy as np

#  Log system        
class PS_Search_Color:

    def color_detection(self, frame_in):
        # Take each frame
        #frame = cv2.imread(image_name)
        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / 4)
        roiWidth = roiX * 2
        roiY = src_height / 4
        roiHeight = roiY * 2
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # detect red
        lower = np.array([150, 30, 30])
        upper = np.array([190, 255, 255])
        mask1 = cv2.inRange(hsv, lower, upper)
        lower = np.array([0, 30, 30])
        upper = np.array([10, 255, 255])
        mask2 = cv2.inRange(hsv, lower, upper)
        mask = mask1 + mask2
        redVal = float(mask.sum()) / float(max_value)
        if redVal > 0.4:
            red=round(redVal,2)
        else:
            red=round(redVal,2)

        # detect yellow
        lower = np.array([5, 100, 100])
        upper = np.array([40, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        yellowVal = float(mask.sum()) / float(max_value)
        if yellowVal > 0.15:
            yellow=round(yellowVal,2)
        else:
            yellow=round(yellowVal,2)

        # detect blue
        lower = np.array([100, 60, 60])
        upper = np.array([140, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        blueVal = float(mask.sum()) / float(max_value)
        if blueVal > 0.5:
            blue=round(blueVal,2)
        else:
            blue=round(blueVal,2)

        # detect green (gray)
        lower_green = np.array([103, 86, 65])
        upper_green = np.array([145, 133, 128])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        greenVal = float(mask.sum()) / float(max_value)
        if greenVal > 0.01:
            green=round(greenVal,2)
        else:
            green=round(greenVal,2)

        # detect white
        lower = np.array([0, 0, 140])
        upper = np.array([256, 60, 256])
        mask = cv2.inRange(hsv, lower, upper)
        whiteVal = float(mask.sum()) / float(max_value)
        if whiteVal > 0.5:
            white=round(whiteVal,2)
        else:
            white=round(whiteVal,2)

        # detect black
        lower_black = np.array([110,50,50])
        upper_black= np.array([130,255,255])
        mask = cv2.inRange(hsv, lower_black, upper_black)
        blackVal = float(mask.sum()) / float(max_value)
        if blackVal > 0.15:
            black=round(blackVal,2)
        else:
            black=round(blackVal,2)

        return red, yellow, blue, green, white, black



In [2]:
#white balance
import numpy as np
from PIL import Image
import sys
import matplotlib.pyplot as plt

%matplotlib inline

def from_pil(pimg):
    
    pimg = cv2.cvtColor(pimg, cv2.COLOR_BGR2RGB)
    pimg = Image.fromarray(pimg)
    pimg = pimg.convert(mode='RGB')
    nimg = np.asarray(pimg)
    nimg.flags.writeable = True
    return nimg

def to_pil(nimg):
    


    #return Image.fromarray(np.uint8(nimg))
    rgb_nimg = Image.fromarray(np.uint8(nimg)).convert('RGB')
    opencv_img = np.array(rgb_nimg)
    # Convert RGB to BGR
    opencv_img = opencv_img[:, :, ::-1].copy()
    return opencv_img

def retinex(nimg):
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = nimg[1].max()
    nimg[0] = np.minimum(nimg[0]*(mu_g/float(nimg[0].max())),255)
    nimg[2] = np.minimum(nimg[2]*(mu_g/float(nimg[2].max())),255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

def retinex_adjust(nimg):
    """
    from 'Combining Gray World and Retinex Theory for Automatic White Balance in Digital Photography'
    """
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    sum_r = np.sum(nimg[0])
    sum_r2 = np.sum(nimg[0]**2)
    max_r = nimg[0].max()
    max_r2 = max_r**2
    sum_g = np.sum(nimg[1])
    max_g = nimg[1].max()
    coefficient = np.linalg.solve(np.array([[sum_r2,sum_r],[max_r2,max_r]]),
                                  np.array([sum_g,max_g]))

    nimg[0] = np.minimum((nimg[0]**2)*coefficient[0] + nimg[0]*coefficient[1],255)
    sum_b = np.sum(nimg[1])
    sum_b2 = np.sum(nimg[1]**2)
    max_b = nimg[1].max()
    max_b2 = max_r**2
    coefficient = np.linalg.solve(np.array([[sum_b2,sum_b],[max_b2,max_b]]),
                                             np.array([sum_g,max_g]))
    nimg[1] = np.minimum((nimg[1]**2)*coefficient[0] + nimg[1]*coefficient[1],255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

def imgFilt(img,filtratio):
    ratio = filtratio*255
    #filtered_img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    filtered_img = img
    grimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2GRAY)
    
    #hsvimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2HSV)
        
    lightMask = np.asarray(grimg)
    Mask_indices = lightMask > ratio
    filtered_img[Mask_indices] =255
    return filtered_img,Mask_indices

def detectDark(img,rat):
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    hei,wid,chan = hsv.shape
    ratio = np.sum(v)/(hei*wid*255)
    if(ratio < rat):
        return True
    else:
        return False
            



In [3]:
{'red_h_lower': 17.28268801461374,
 'red_h_upper': 53.92051246764519,
 'red_s_lower': 109.33589366491391,
 'threshold': 3.343668404254858}

color_boundaries_bank = {
    'red': ([149, 20, 44],[187, 255, 255]), 
    'yellow': ([ 14, 40, 46],[ 32, 255, 255]),
    'blue':([ 71, 57, 99],[116, 255, 255]), 
    'green':([ 42, 26, 41],[ 99, 255, 255]),
    'white':([  0,  0, 197],[179,  7, 255],[ 0, 7,173],[179, 44,255]),
    'black':([  0,  0,  0],[179,  17, 53],[ 0, 17,  0],[179,255, 109])
}

# define base threshold, default 0.3
color_baseThreshold = {
    'red': 0.22, #daytime
    #'red': 0.1, #night
    'yellow': 0.14297297267509484,#daytime
    'blue':0.35, #daytime
    'green':0.2, #daytime
    'white':0.3261858048268709,
    'black':0.2671834202127429
}

ps_color_baseThreshold = {
    'red': 0.22, #daytime
    #'red': 0.1, #night
    'yellow': 0.14297297267509484,#daytime
    'blue':0.35, #daytime
    'green':0.2, #daytime
    'white':0.3261858048268709,
    'black':0.24363711095687401
     
}



clr_list = {'red':0,'yellow':1,'blue':2,'green':3,'white':4,'black':5}

import sys
import cv2
import numpy as np

#  Log system        
class DL_Search_Color:

    def color_detection(self, frame_in):
        
        color_to_be_detected = ['red','yellow','blue','green','white','black']
        # Take each frame
        #frame = cv2.imread(image_name)
        frame_in = to_pil(retinex_adjust(retinex(from_pil(frame_in))))
        
        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / 4)
        roiWidth = roiX * 2
        roiY = src_height / 4
        roiHeight = roiY * 2
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        res = [False] * len(color_to_be_detected)
        for idx,color in enumerate(color_to_be_detected):
            color_boundaries = color_boundaries_bank[color]
            assert len(color_boundaries) % 2 == 0, "Each boundary must have a lower/upper pair"

            mask = 0
            #loop over each boundary pair for this color
            for lower,upper in zip(color_boundaries[0::2], color_boundaries[1::2]):
                mask = mask + cv2.inRange(hsv, np.array(lower), np.array(upper))
                if(float(max_value)!=0):
                    Val = float(mask.sum()) / float(max_value)
                    #if Val > color_baseThreshold[color]:
                else:
                    Val = 0
                    
                res[idx] = round(Val,2)
        return res

In [4]:
search_color = DL_Search_Color()
PS_search_color = PS_Search_Color()

In [5]:
def test(color_cls):
    # import the necessary packages
    import numpy as np
    import argparse
    import cv2
    import matplotlib.pyplot as plt
    import sys, os
    import time
    #search_color = PS_Search_Color()
    
    clr_idx = clr_list[color_cls]

    color_to_be_tested = ['red','yellow','blue','green','white','black']

    target_folder = 'color_data/21/'
    this_color_count = 0
    fp = 0
    fn = 0
    ps_fp = 0
    ps_fn = 0
    #ps_count = 0
    total = 0
    for idx, color in enumerate(color_to_be_tested):
        for file in os.listdir(target_folder + color):
            if(file != os.path.basename("__file__")):
                if(file[-3:] == 'jpg'):
            
                    origin_img = cv2.imread(target_folder + color + '/' + file)

            
                    res = search_color.color_detection(origin_img)
                    ps_res = PS_search_color.color_detection(origin_img)
                    
                    
                    total += 1
                    
                    if res[clr_idx]>=color_baseThreshold[color_cls] and clr_idx != idx:
                        fp = fp +1
                    elif clr_idx == idx:
                        this_color_count += 1
                        if res[clr_idx]<color_baseThreshold[color_cls]:
                            fn = fn + 1
                    if ps_res[clr_idx]>=color_baseThreshold[color_cls] and clr_idx != idx:
                        ps_fp = ps_fp +1
                    elif ps_res[clr_idx]<color_baseThreshold[color_cls] and clr_idx == idx:
                        ps_fn = ps_fn + 1
    print("accuracy:",float(fp)/(total-this_color_count) + float(fn)/this_color_count)
    print("total",total)
    print("color :",this_color_count)
    
    print("dl  fp:",fp)
    print("dl  fn:",fn)
    print("ps  fp:",ps_fp)
    print("ps  fn:",ps_fn)
    print('\n')
    return fp,fn

color_to_be_tested = ['red','yellow','blue','green','white','black']
for color in color_to_be_tested:   
    print(color)
    fp,fn = test(color)

red
('accuracy:', 0.3001251172974664)
('total', 1227)
('color :', 115)
('dl  fp:', 92)
('dl  fn:', 25)
('ps  fp:', 316)
('ps  fn:', 1)


yellow
('accuracy:', 0.09212187666135034)
('total', 1227)
('color :', 171)
('dl  fp:', 17)
('dl  fn:', 13)
('ps  fp:', 6)
('ps  fn:', 33)


blue
('accuracy:', 0.49136521112146736)
('total', 1227)
('color :', 101)
('dl  fp:', 7)
('dl  fn:', 49)
('ps  fp:', 29)
('ps  fn:', 38)


green
('accuracy:', 0.3045697617890131)
('total', 1227)
('color :', 17)
('dl  fp:', 155)
('dl  fn:', 3)
('ps  fp:', 36)
('ps  fn:', 12)


white
('accuracy:', 0.2908241160770185)
('total', 1227)
('color :', 503)
('dl  fp:', 22)
('dl  fn:', 131)
('ps  fp:', 49)
('ps  fn:', 26)


black
('accuracy:', 0.4838306229327453)
('total', 1227)
('color :', 320)
('dl  fp:', 436)
('dl  fn:', 1)
('ps  fp:', 50)
('ps  fn:', 297)




In [5]:
color_boundaries_bank = {
    'red': ([149, 20, 44],[187, 255, 255]), 
    'yellow': ([ 14, 40, 46],[ 32, 255, 255]),
    'blue':([ 71, 57, 99],[116, 255, 255]), 
    'green':([ 42, 26, 41],[ 99, 255, 255]),
    'white':([  0,  0, 197],[179,  7, 255],[ 0, 7,173],[179, 44,255]),
    'black':([  0,  0,  0],[179,  17, 53],[ 0, 17,  0],[179,255, 109])
}


#  Log system        
class test_Search_Color:

    def color_detection(self, frame_in, arg1, arg2, arg3, arg4, arg5):
        
        #color_to_be_detected = ['red','yellow','blue','green','white','black']
        # Take each frame
        #frame = cv2.imread(image_name)
        
        frame_in = to_pil(retinex_adjust(retinex(from_pil(frame_in))))
        
        roiWid = 10
        roiEdg = arg1
        
        
        src_height, src_width, src_channels = frame_in.shape
        
        roiX = int(src_width / roiWid)
        roiWidth = roiX * roiEdg
        roiY = int(src_height / roiWid)
        roiHeight = roiY * roiEdg
        
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        

        
        frame,lightMask = imgFilt(frame,arg2)

        src_height, src_width, src_channels = frame.shape
        blockedPxl = sum(np.count_nonzero(e) for e in lightMask)
        max_value = ((src_height*src_width)-blockedPxl)*255

        #src_height, src_width, src_channels = frame.shape
        #max_value = src_height * src_width * 255
        
    
        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        """
        # red [149, 20, 44],[187, 255, 255]
        lower = np.array([int(arg3),int(arg4), int(arg5)])   
        upper = np.array([int(arg6), 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        """
        
        """
        # white [  0,  0, 197],[179,  7, 255],[ 0, 7,173],[179, 44,255]
        lower = np.array([0,0, int(arg1)])   
        upper = np.array([179, int(arg2), 255])
        mask = cv2.inRange(hsv, lower, upper)
        lower = np.array([0,int(arg2), int(arg3)])
        upper = np.array([179, int(arg4), 255])
        mask = mask + cv2.inRange(hsv, lower, upper)
        
        """
        # black
        lower = np.array([0,0, 0])   
        upper = np.array([179, int(arg3), int(arg4)])
        mask = cv2.inRange(hsv, lower, upper)
        lower = np.array([0,int(arg3), 0])
        upper = np.array([179, 255, int(arg5)])
        mask = mask + cv2.inRange(hsv, lower, upper)
        
        
        #[  0,  0,  0],[179,  100, 100],[ 0, 100,  0],[179,255, 50]
        if(float(max_value) != 0):
            Val = float(mask.sum()) / float(max_value)
        else:
            Val = 0
    
        res = round(Val,2)


        return res

In [6]:
test_search_color = test_Search_Color()

In [7]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import sys, os
import time
#search_color = PS_Search_Color()
def test(color_cls,arg1,arg2,arg3,arg4,arg5,arg6):

    clr_idx = clr_list[color_cls]

    color_to_be_tested = ['red','yellow','blue','green','white','black']
    target_folder = 'color_data/21/'
    this_color_count = 0
    fp = 0
    fn = 0
    #ps_count = 0
    total = 0
    for idx, color in enumerate(color_to_be_tested):
        for file in os.listdir(target_folder + color):
            if(file != os.path.basename("__file__")):
                if(file[-3:] == 'jpg'):
                    origin_img = cv2.imread(target_folder + color + '/' + file)
                    #print(origin_img)
                    #plt.figure()
                    #plt.imshow(origin_img)
            
                    #plt.show()
                    #print('red yellow blue green white black')
                    #print(search_color.color_detection(origin_img))
                    #print(search_color.color_detection(origin_img))
                    res = test_search_color.color_detection(origin_img,arg2,arg3,arg4,arg5,arg6)
                    #ps_res = PS_search_color.color_detection(origin_img)E
                    
                    total += 1
                    
                    if res>=arg1 and clr_idx != idx:
                        fp = fp +1
                    elif clr_idx == idx:
                        this_color_count += 1
                        if res<arg1:
                            fn = fn + 1

    return total,this_color_count,fp,fn



In [66]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pprint import pprint
from hyperopt import hp
import hyperopt.pyll.stochastic
from __future__ import division

# minimize the objective over the space
from hyperopt import fmin, tpe
import hyperopt
import sys
import time

def restart_line():
    sys.stdout.write('\r')
    sys.stdout.flush()


min = 1
state = ''
# define an objective function
def objective(args):
    global min
    global state
    
    threshold, roiScale, filtratio, \
    black_h_lower, black_h_upper, black_s_lower=args#, #white_v_lower = args
    #blue_h_lower, blue_h_upper, blue_s_lower, blue_v_lower = args
    #yellow_h_lower, yellow_h_upper, yellow_s_lower, yellow_v_lower = args
    #red_h_lower,red_h_upper,red_s_lower, red_v_lower = args
    
    # green_h_lower, green_h_upper, green_s_lower, green_v_lower = args
    # 
    
    
    thr = 0.1 + threshold * 0.05
    
    total,this_color_count,fp,fn = test('black',thr, roiScale, filtratio, \
                                        #red_h_lower, red_h_upper , red_s_lower, red_v_lower)
                                        #yellow_h_lower, yellow_h_upper, yellow_s_lower, yellow_v_lower)
                                        #blue_h_lower, blue_h_upper, blue_s_lower, blue_v_lower)
                                        black_h_lower, black_h_upper, black_s_lower)#, white_v_lower)
                                        #
                    
    tp = total - fp - fn
    beta = 3
    print(total,this_color_count)
    
    f1_score = float(((1 + beta**2) * tp)) / ((1 + beta**2)*tp + beta**2 * fn + fp)
    if(1 - f1_score < min):
        min = 1 - f1_score
        state = 'f1_score: ' + str(round(1-f1_score,3)) + ', fp: ' + str(round(fp/(total-this_color_count),3)) + \
        ', fn: ' + str(round(fn/this_color_count,3)) + '\n' + \
        'thr: ' + str(thr) + '\n' + 'roiScale: ' + str(roiScale)+ '\n' + 'filtratio: ' +  str(filtratio)+ '\n'\
        + 'black_h_lower: ' +  str(black_h_lower)\
        + '\n'+ 'black_h_upper: ' + str(black_h_upper) + '\n'+ 'black_s_lower: ' + str(black_s_lower)+ '\n' 
        #'white_v_lower: ' + str(white_v_lower) + '\n'
        print(state)
        


    return {
        'loss': 1-f1_score,
        'status': STATUS_OK,
        'eval_time':time.time()
    }
 
space = [  
         hp.uniform('threshold',0,10), #threshold of detection      
         hp.uniform('roiScale',4,10), #roi scale   
         hp.uniform('filtratio',0.5,0.75),  #mask filter threshold

         hp.uniform('black_h_lower',15,45),hp.uniform('black_h_upper',50,80),
         hp.uniform('black_s_lower', 60,120)
         
         
]
        
    
"""
        #red   
         hp.uniform('red_h_lower',140,160),hp.uniform('red_h_upper',20,60),
         hp.uniform('red_s_lower', 30,70),hp.uniform('red_v_lower',175,195)
        yellow
        hp.uniform('yellow_h_lower',0,25),hp.uniform('yellow_h_upper',20,60),
         hp.uniform('yellow_s_lower', 30,70),hp.uniform('yellow_v_lower',25,50),

         blue
         hp.uniform('blue_h_lower',70,95),hp.uniform('blue_h_upper',35,75),
         hp.uniform('blue_s_lower', 30,80),hp.uniform('blue_v_lower',115,145),
         green
         hp.uniform('green_h_lower',25,45),hp.uniform('green_h_upper',35,75),
         hp.uniform('green_s_lower', 30,80),hp.uniform('green_v_lower',75,100),
         white
         hp.uniform('white_h_lower',170,210),hp.uniform('white_h_upper',0,20),
         hp.uniform('white_s_lower', 160,210),hp.uniform('white_v_lower',15,60),
         black
         hp.uniform('black_h_lower',15,45),hp.uniform('black_h_upper',50,80),
         hp.uniform('black_s_lower', 50,120)#hp.uniform('red_v_lower',15,45),
"""   



#pprint(trials.trials)
print('\n')


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=1000,trials= trials)


pprint(best)
print('\n')

# -> {'a': 1, 'c2': 0.01420615366247227}
pprint(trials.losses())
#pprint(hyperopt.space_eval(space, best))
print('\n')
# -> ('case 2', 0.01420615366247227}

/home/max/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(1227, 320)
f1_score: 0.081, fp: 0.484, fn: 0.081
thr: 0.443027375587
roiScale: 4.19299970656
filtratio: 0.63395849007
black_h_lower: 18.1015019562
black_h_upper: 55.4771579145
black_s_lower: 89.9815578224

(1227, 320)
f1_score: 0.07, fp: 0.031, fn: 0.281
thr: 0.545474273685
roiScale: 7.20773542881
filtratio: 0.672898178393
black_h_lower: 34.9816909039
black_h_upper: 74.957498254
black_s_lower: 67.3599419753

(1227, 320)
(1227, 320)
f1_score: 0.053, fp: 0.138, fn: 0.163
thr: 0.3327900931
roiScale: 7.70595753531
filtratio: 0.726318615041
black_h_lower: 28.3220903987
black_h_upper: 76.6445073927
black_s_lower: 63.6352380897

(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
(1227, 320)
f1_score: 0.035, fp: 0.246, fn: 0.047
thr: 0.515204471349
roiScale: 6.89098467699
filtratio: 0.688433927959
black_h_lower: 22.3781326971
black_h_upper: 73.1654248964
black_s_lower: 100.48340853

(1227,

In [ ]:
#red
f1_score: 0.03, fp: 0.176, fn: 0.252
thr: 0.167891853142
roiScale: 5.85866266622
filtratio: 0.541101528153
red_h_lower: 143.648775041
red_h_upper: 35.1614769342
red_s_lower: 46.9720000947
red_v_lower: 183.622530423

f1_score: 0.027, fp: 0.2, fn: 0.217
thr: 0.119282447213
roiScale: 5.25134749062
filtratio: 0.568460169578
red_h_lower: 140.534398321
red_h_upper: 31.0692469852
red_s_lower: 65.4059626521
red_v_lower: 180.386850776

f1_score: 0.025, fp: 0.153, fn: 0.217
thr: 0.179285638177
roiScale: 7.6085589874
filtratio: 0.547474296985
red_h_lower: 144.004234376
red_h_upper: 20.6063288116
red_s_lower: 50.0373385571
red_v_lower: 185.074896369

f1_score: 0.024, fp: 0.139, fn: 0.217
thr: 0.170089231256
roiScale: 7.80363785125
filtratio: 0.55884392388
red_h_lower: 144.050882973
red_h_upper: 20.9695747113
red_s_lower: 51.9970235561
red_v_lower: 185.50535228

f1_score: 0.018, fp: 0.26, fn: 0.122
thr: 0.100615081421
roiScale: 6.92424965687
filtratio: 0.63647201966
red_h_lower: 149.636466985
red_h_upper: 30.9586824884
red_s_lower: 32.7783603598
red_v_lower: 193.387021405

f1_score: 0.017, fp: 0.188, fn: 0.13
thr: 0.108198360014
roiScale: 7.06246294758
filtratio: 0.678560793345
red_h_lower: 159.262768207
red_h_upper: 29.8553567453
red_s_lower: 31.5860732627
red_v_lower: 193.46382715

f1_score: 0.016, fp: 0.242, fn: 0.113
thr: 0.100636131884
roiScale: 6.7643157437
filtratio: 0.632523025803
red_h_lower: 159.696599532
red_h_upper: 21.2677222425
red_s_lower: 31.0373056996
red_v_lower: 189.281075252

f1_score: 0.016, fp: 0.171, fn: 0.13
thr: 0.106741019408
roiScale: 8.10936384066
filtratio: 0.678846251785
red_h_lower: 156.034750678
red_h_upper: 27.2220421891
red_s_lower: 44.0905590051
red_v_lower: 194.214364959

f1_score: 0.016, fp: 0.227, fn: 0.113
thr: 0.101314828791
roiScale: 6.59687882644
filtratio: 0.707729980832
red_h_lower: 157.566940978
red_h_upper: 26.9842204292
red_s_lower: 31.5788460783
red_v_lower: 193.314803362

f1_score: 0.016, fp: 0.259, fn: 0.104
thr: 0.109874267792
roiScale: 6.33610807515
filtratio: 0.680823689267
red_h_lower: 155.516557224
red_h_upper: 22.3794223858
red_s_lower: 41.655642215
red_v_lower: 186.362897322

f1_score: 0.015, fp: 0.199, fn: 0.113
thr: 0.10846576358
roiScale: 6.58922945039
filtratio: 0.711146236999
red_h_lower: 159.124367903
red_h_upper: 27.0129842965
red_s_lower: 33.4312479383
red_v_lower: 188.481745159

f1_score: 0.015, fp: 0.198, fn: 0.113
thr: 0.100199422492
roiScale: 6.64650042546
filtratio: 0.749478069867
red_h_lower: 159.094213726
red_h_upper: 26.1147503503
red_s_lower: 36.0768169611
red_v_lower: 187.957113392

f1_score: 0.015, fp: 0.194, fn: 0.113
thr: 0.10031004576
roiScale: 6.68859857987
filtratio: 0.746913575017
red_h_lower: 158.646572979
red_h_upper: 28.4884717166
red_s_lower: 36.2722467711
red_v_lower: 188.334957068

f1_score: 0.014, fp: 0.17, fn: 0.113
thr: 0.100253430667
roiScale: 7.48373645434
filtratio: 0.749497430985
red_h_lower: 156.751890357
red_h_upper: 31.0274401421
red_s_lower: 30.6353992628
red_v_lower: 188.208709365

{'filtratio': 0.7494974309846314,
 'red_h_lower': 156.7518903571603,
 'red_h_upper': 31.027440142139483,
 'red_s_lower': 30.635399262835616,
 'red_v_lower': 188.2087093651904,
 'roiScale': 7.483736454337399,
 'threshold': 0.005068613347630142}

In [ ]:
#yellow
f1_score: 0.081, fp: 0.01, fn: 0.579
thr: 0.291049301705
roiScale: 8.91656138137
filtratio: 0.664882013796
red_h_lower: 2.28952243249
red_h_upper: 59.6607530545
red_s_lower: 38.6689580893
red_v_lower: 34.4069517625

f1_score: 0.053, fp: 0.007, fn: 0.38
thr: 0.302937524543
roiScale: 8.75001742581
filtratio: 0.699464275935
red_h_lower: 22.8526181099
red_h_upper: 35.0708512078
red_s_lower: 39.7119733053
red_v_lower: 49.3634526444

f1_score: 0.029, fp: 0.034, fn: 0.205
thr: 0.196499501287
roiScale: 6.9558028453
filtratio: 0.665608795421
red_h_lower: 24.1762063703
red_h_upper: 21.8230198567
red_s_lower: 38.6415656554
red_v_lower: 46.4330741249

f1_score: 0.025, fp: 0.093, fn: 0.164
thr: 0.121598004853
roiScale: 4.65344788392
filtratio: 0.636485349557
red_h_lower: 16.8194697207
red_h_upper: 27.8472259202
red_s_lower: 69.2358894098
red_v_lower: 44.7800085442

f1_score: 0.018, fp: 0.08, fn: 0.117
thr: 0.100269139199
roiScale: 5.39745711692
filtratio: 0.624017710542
red_h_lower: 17.2383351436
red_h_upper: 30.2513699232
red_s_lower: 69.4791684378
red_v_lower: 45.9222983862

f1_score: 0.006, fp: 0.046, fn: 0.041
thr: 0.114639003707
roiScale: 6.2530020193
filtratio: 0.677256307924
red_h_lower: 15.5312466325
red_h_upper: 43.8330308854
red_s_lower: 61.9263462159
red_v_lower: 49.1795576147

f1_score: 0.004, fp: 0.082, fn: 0.023
thr: 0.124727469568
roiScale: 7.11552465145
filtratio: 0.728873067542
red_h_lower: 8.42292727858
red_h_upper: 42.2809366578
red_s_lower: 48.9627181607
red_v_lower: 44.4824112703

f1_score: 0.004, fp: 0.121, fn: 0.018
thr: 0.101315882548
roiScale: 8.16724974073
filtratio: 0.731849276537
red_h_lower: 6.85242977587
red_h_upper: 46.0416490932
red_s_lower: 47.7579728553
red_v_lower: 44.8889092758

f1_score: 0.004, fp: 0.1, fn: 0.018
thr: 0.100172937964
roiScale: 7.10430461391
filtratio: 0.733229458418
red_h_lower: 7.18318803359
red_h_upper: 47.0789595442
red_s_lower: 52.3188982914
red_v_lower: 47.2246818678

f1_score: 0.003, fp: 0.064, fn: 0.018
thr: 0.10957739751
roiScale: 7.84679940326
filtratio: 0.749995571735
red_h_lower: 10.9676001777
red_h_upper: 45.1153420635
red_s_lower: 64.9628544817
red_v_lower: 48.8864279701

f1_score: 0.003, fp: 0.123, fn: 0.012
thr: 0.119362640588
roiScale: 7.73402636987
filtratio: 0.738887879875
red_h_lower: 8.39542614061
red_h_upper: 34.3938858722
red_s_lower: 57.6692729775
red_v_lower: 49.96268018

f1_score: 0.003, fp: 0.101, fn: 0.012
thr: 0.123648491987
roiScale: 7.90592404447
filtratio: 0.749350323374
red_h_lower: 8.9781448217
red_h_upper: 33.0607676632
red_s_lower: 62.5539232872
red_v_lower: 48.3805418946

f1_score: 0.003, fp: 0.1, fn: 0.012
thr: 0.100497797252
roiScale: 7.78850296228
filtratio: 0.716028315261
red_h_lower: 9.87939925359
red_h_upper: 36.8118379219
red_s_lower: 66.2932367569
red_v_lower: 49.5058582042

f1_score: 0.003, fp: 0.09, fn: 0.012
thr: 0.100793423285
roiScale: 7.56365097706
filtratio: 0.732794672118
red_h_lower: 9.496004931
red_h_upper: 40.7645982214
red_s_lower: 64.3251760505
red_v_lower: 49.6185009858

f1_score: 0.003, fp: 0.089, fn: 0.012
thr: 0.100278980749
roiScale: 7.58682327987
filtratio: 0.73268050347
red_h_lower: 10.6906389279
red_h_upper: 39.5087970993
red_s_lower: 63.5473656696
red_v_lower: 48.6221800817

f1_score: 0.003, fp: 0.087, fn: 0.012
thr: 0.110518929129
roiScale: 7.89983722026
filtratio: 0.749928768266
red_h_lower: 10.2633381293
red_h_upper: 35.5287138601
red_s_lower: 65.3579433688
red_v_lower: 49.6303260267

f1_score: 0.003, fp: 0.086, fn: 0.012
thr: 0.110828019521
roiScale: 8.10226540372
filtratio: 0.726779208663
red_h_lower: 17.7191314041
red_h_upper: 26.2725118805
red_s_lower: 62.5517803665
red_v_lower: 49.6220248598

f1_score: 0.002, fp: 0.071, fn: 0.012
thr: 0.11154491417
roiScale: 8.04552428634
filtratio: 0.733320364867
red_h_lower: 20.6835988849
red_h_upper: 24.509801927
red_s_lower: 62.0430596383
red_v_lower: 49.1253454984

f1_score: 0.002, fp: 0.051, fn: 0.012
thr: 0.100105193296
roiScale: 8.00571013085
filtratio: 0.738486872701
red_h_lower: 21.9118627512
red_h_upper: 28.2174182863
red_s_lower: 64.0659716274
red_v_lower: 48.1851920485

f1_score: 0.002, fp: 0.05, fn: 0.012
thr: 0.109791312935
roiScale: 7.86965071386
filtratio: 0.744969997962
red_h_lower: 19.5975726566
red_h_upper: 25.62420642
red_s_lower: 69.8350293139
red_v_lower: 37.985579032

f1_score: 0.002, fp: 0.045, fn: 0.012
thr: 0.10075361658
roiScale: 8.05507155667
filtratio: 0.733022641538
red_h_lower: 20.5710470333
red_h_upper: 24.8838188052
red_s_lower: 67.883163262
red_v_lower: 38.035554283

f1_score: 0.002, fp: 0.043, fn: 0.012
thr: 0.100264885893
roiScale: 7.76412718615
filtratio: 0.72334700927
red_h_lower: 20.4636441867
red_h_upper: 23.7250339541
red_s_lower: 67.4734741128
red_v_lower: 35.7809186882

{'filtratio': 0.723347009269909,
 'roiScale': 7.764127186149392,
 'threshold': 0.0052977178558566256,
 'yellow_h_lower': 20.463644186708464,
 'yellow_h_upper': 23.7250339541127,
 'yellow_s_lower': 67.47347411284402,
 'yellow_v_lower': 35.78091868823839}

In [ ]:
f1_score: 0.08, fp: 0.145, fn: 0.832
thr: 0.159564009032
roiScale: 7.47062279306
filtratio: 0.625111403773
blue_h_lower: 82.2736055372
blue_h_upper: 48.9396774501
blue_s_lower: 41.4996868979
blue_v_lower: 106.037115525

f1_score: 0.035, fp: 0.355, fn: 0.248
thr: 0.17234650928
roiScale: 5.45270070108
filtratio: 0.657760007056
blue_h_lower: 87.7224926861
blue_h_upper: 33.0810904004
blue_s_lower: 51.3731099638
blue_v_lower: 119.166660726

f1_score: 0.012, fp: 0.285, fn: 0.079
thr: 0.163104898079
roiScale: 9.33923933325
filtratio: 0.552572899408
blue_h_lower: 78.9192808741
blue_h_upper: 50.9339024968
blue_s_lower: 51.7856971561
blue_v_lower: 115.251498667

f1_score: 0.011, fp: 0.291, fn: 0.069
thr: 0.240023419466
roiScale: 8.50048868726
filtratio: 0.661779865642
blue_h_lower: 85.6516266075
blue_h_upper: 25.0235049854
blue_s_lower: 68.1841578055
blue_v_lower: 117.932751886

f1_score: 0.006, fp: 0.366, fn: 0.01
thr: 0.101452965372
roiScale: 9.21114148957
filtratio: 0.649536756753
blue_h_lower: 79.6736166794
blue_h_upper: 55.8877920425
blue_s_lower: 42.1790461089
blue_v_lower: 117.210410268

f1_score: 0.006, fp: 0.195, fn: 0.04
thr: 0.13728302415
roiScale: 9.07804020399
filtratio: 0.720145326397
blue_h_lower: 80.2931179893
blue_h_upper: 59.5693648315
blue_s_lower: 45.2131464089
blue_v_lower: 116.799466489

f1_score: 0.005, fp: 0.195, fn: 0.03
thr: 0.140069654249
roiScale: 9.14568117299
filtratio: 0.726694222026
blue_h_lower: 80.6899156155
blue_h_upper: 59.3537544528
blue_s_lower: 46.9464080833
blue_v_lower: 119.958902257

f1_score: 0.003, fp: 0.079, fn: 0.03
thr: 0.183862907852
roiScale: 8.17531367096
filtratio: 0.680364171828
blue_h_lower: 83.0082914734
blue_h_upper: 53.6009285786
blue_s_lower: 64.1325115706
blue_v_lower: 119.233235486

f1_score: 0.003, fp: 0.077, fn: 0.03
thr: 0.148646042596
roiScale: 7.80545628644
filtratio: 0.720573566344
blue_h_lower: 82.7647376388
blue_h_upper: 56.1088489864
blue_s_lower: 68.9264021326
blue_v_lower: 119.420092668

f1_score: 0.002, fp: 0.152, fn: 0.0
thr: 0.129476830898
roiScale: 8.16989197732
filtratio: 0.718071983279
blue_h_lower: 81.1674844237
blue_h_upper: 57.0153918042
blue_s_lower: 59.4408200331
blue_v_lower: 118.384929552

In [ ]:
f1_score: 0.016, fp: 0.092, fn: 1.0
thr: 0.457389700556
roiScale: 9.71487199468
filtratio: 0.549211353141
green_h_lower: 18.1899194259
green_h_upper: 33.7507925318
green_s_lower: 36.7464694488
green_v_lower: 98.84301671

f1_score: 0.004, fp: 0.054, fn: 0.235
thr: 0.261114238465
roiScale: 4.66078431894
filtratio: 0.582389898851
green_h_lower: 24.357537551
green_h_upper: 59.053693143
green_s_lower: 63.4546328448
green_v_lower: 104.451805805

f1_score: 0.004, fp: 0.159, fn: 0.118
thr: 0.223635350207
roiScale: 4.68075854445
filtratio: 0.637718138357
green_h_lower: 32.9057517348
green_h_upper: 48.2765119866
green_s_lower: 33.0894530698
green_v_lower: 109.830105551

f1_score: 0.003, fp: 0.142, fn: 0.118
thr: 0.327218069337
roiScale: 6.76821773598
filtratio: 0.504153835029
green_h_lower: 28.869506609
green_h_upper: 27.6860163131
green_s_lower: 56.8175419896
green_v_lower: 107.134765404

f1_score: 0.003, fp: 0.029, fn: 0.176
thr: 0.269127317364
roiScale: 6.43716342588
filtratio: 0.616033586808
green_h_lower: 31.3951877275
green_h_upper: 54.6574430571
green_s_lower: 49.8023219378
green_v_lower: 105.669402468

f1_score: 0.002, fp: 0.107, fn: 0.059
thr: 0.180002836847
roiScale: 5.03318328962
filtratio: 0.601528097857
green_h_lower: 27.7811021209
green_h_upper: 56.887624769
green_s_lower: 49.0859741792
green_v_lower: 105.921068206

f1_score: 0.002, fp: 0.027, fn: 0.118
thr: 0.229764324981
roiScale: 5.09188342063
filtratio: 0.647370001018
green_h_lower: 32.3180877035
green_h_upper: 50.7297567163
green_s_lower: 60.192374511
green_v_lower: 100.742548613

f1_score: 0.002, fp: 0.016, fn: 0.118
thr: 0.271620113443
roiScale: 6.11650660802
filtratio: 0.660455907974
green_h_lower: 33.1580722551
green_h_upper: 47.9071139001
green_s_lower: 59.7537066122
green_v_lower: 105.093730233

f1_score: 0.002, fp: 0.011, fn: 0.118
thr: 0.268356761611
roiScale: 5.28531225024
filtratio: 0.653618255965
green_h_lower: 33.1138982202
green_h_upper: 58.9713815924
green_s_lower: 59.8837265589
green_v_lower: 104.91671526

f1_score: 0.002, fp: 0.008, fn: 0.118
thr: 0.268243785846
roiScale: 5.31184741799
filtratio: 0.677137177153
green_h_lower: 33.376932604
green_h_upper: 58.8889405153
green_s_lower: 63.5604720849
green_v_lower: 104.955931255

f1_score: 0.002, fp: 0.068, fn: 0.059
thr: 0.159238471805
roiScale: 5.91019361991
filtratio: 0.674400752293
green_h_lower: 30.6621790984
green_h_upper: 58.2302761011
green_s_lower: 50.5397041308
green_v_lower: 100.317825483

f1_score: 0.001, fp: 0.069, fn: 0.0
thr: 0.144859012079
roiScale: 6.1793908063
filtratio: 0.671782928312
green_h_lower: 31.7172440372
green_h_upper: 46.1466242369
green_s_lower: 64.122226636
green_v_lower: 100.3382037

In [ ]:
f1_score: 0.15, fp: 0.185, fn: 0.348
thr: 0.179288138355
roiScale: 5.45920341714
filtratio: 0.749454504429
white_h_lower: 189.124884634
white_h_upper: 16.1831275188
white_s_lower: 188.472597284
white_v_lower: 26.6494879843

f1_score: 0.119, fp: 0.503, fn: 0.012
thr: 0.101643032494
roiScale: 6.7035667123
filtratio: 0.612403583036
white_h_lower: 186.971405667
white_h_upper: 16.9266919838
white_s_lower: 187.102639293
white_v_lower: 26.169761067

f1_score: 0.083, fp: 0.298, fn: 0.064
thr: 0.479364636586
roiScale: 6.75374716898
filtratio: 0.502097822112
white_h_lower: 203.269378041
white_h_upper: 11.2911856597
white_s_lower: 173.09297919
white_v_lower: 21.9190305706

f1_score: 0.079, fp: 0.262, fn: 0.078
thr: 0.511817795755
roiScale: 9.29591744745
filtratio: 0.527869585793
white_h_lower: 197.555832516
white_h_upper: 17.881807928
white_s_lower: 166.003883982
white_v_lower: 52.3430243671

f1_score: 0.075, fp: 0.326, fn: 0.02
thr: 0.263709452158
roiScale: 8.42734087653
filtratio: 0.568366937085
white_h_lower: 202.99905856
white_h_upper: 16.910405602
white_s_lower: 165.925124458
white_v_lower: 36.1349376545

f1_score: 0.073, fp: 0.28, fn: 0.046
thr: 0.343739219382
roiScale: 8.69247270539
filtratio: 0.564735068966
white_h_lower: 197.477673503
white_h_upper: 19.512018731
white_s_lower: 167.972993368
white_v_lower: 43.2647090114

f1_score: 0.071, fp: 0.265, fn: 0.052
thr: 0.344690520219
roiScale: 8.66710714231
filtratio: 0.584096863153
white_h_lower: 196.82545111
white_h_upper: 19.7844085548
white_s_lower: 168.366453225
white_v_lower: 41.9661620573

f1_score: 0.07, fp: 0.26, fn: 0.052
thr: 0.358395836336
roiScale: 8.14510723438
filtratio: 0.590552162886
white_h_lower: 204.897567537
white_h_upper: 14.7560809643
white_s_lower: 184.585916814
white_v_lower: 46.4988469721

f1_score: 0.069, fp: 0.256, fn: 0.054
thr: 0.343887723009
roiScale: 8.70320434064
filtratio: 0.595565546082
white_h_lower: 200.556689688
white_h_upper: 5.21239038876
white_s_lower: 163.227822775
white_v_lower: 45.4822515933

f1_score: 0.069, fp: 0.253, fn: 0.054
thr: 0.386351818461
roiScale: 8.32737184933
filtratio: 0.580768251548
white_h_lower: 202.422467688
white_h_upper: 6.54337644365
white_s_lower: 162.995071492
white_v_lower: 16.8638791284

In [ ]:
f1_score: 0.068, fp: 0.407, fn: 0.081
thr: 0.441183436814
roiScale: 5.73581832979
filtratio: 0.554255699525
black_h_lower: 25.2265984062
black_h_upper: 67.8114249885
black_s_lower: 79.1980253384

f1_score: 0.046, fp: 0.373, fn: 0.031
thr: 0.536868216812
roiScale: 4.92123989649
filtratio: 0.729733432625
black_h_lower: 22.5416891482
black_h_upper: 71.5788580717
black_s_lower: 112.657938446

f1_score: 0.035, fp: 0.106, fn: 0.106
thr: 0.5928222414
roiScale: 8.20968129064
filtratio: 0.702742101547
black_h_lower: 27.3524420306
black_h_upper: 78.6983401661
black_s_lower: 98.945311986

f1_score: 0.035, fp: 0.109, fn: 0.103
thr: 0.588750955028
roiScale: 8.20399375852
filtratio: 0.699772115055
black_h_lower: 29.1242951646
black_h_upper: 79.4364851682
black_s_lower: 97.518873807

f1_score: 0.031, fp: 0.138, fn: 0.075
thr: 0.590870272592
roiScale: 7.96830150781
filtratio: 0.712228185126
black_h_lower: 28.382321148
black_h_upper: 76.7651615668
black_s_lower: 105.867974885

f1_score: 0.026, fp: 0.233, fn: 0.022
thr: 0.531258798476
roiScale: 7.23973675714
filtratio: 0.742944380985
black_h_lower: 17.6612797381
black_h_upper: 63.2867017006
black_s_lower: 110.644068031

f1_score: 0.026, fp: 0.164, fn: 0.047
thr: 0.597532153056
roiScale: 6.76302019789
filtratio: 0.745340828541
black_h_lower: 18.3042035718
black_h_upper: 58.4118910887
black_s_lower: 110.787013836

f1_score: 0.026, fp: 0.202, fn: 0.031
thr: 0.588859858587
roiScale: 7.28205853252
filtratio: 0.749918187599
black_h_lower: 15.5292621598
black_h_upper: 52.2949860748
black_s_lower: 114.702525519

f1_score: 0.025, fp: 0.207, fn: 0.028
thr: 0.580443011033
roiScale: 6.84921204316
filtratio: 0.735722802954
black_h_lower: 15.343033519
black_h_upper: 52.0940340029
black_s_lower: 112.701903455

f1_score: 0.025, fp: 0.178, fn: 0.037
thr: 0.561865383013
roiScale: 6.96105818037
filtratio: 0.745931995517
black_h_lower: 15.4546814088
black_h_upper: 57.2082327167
black_s_lower: 107.085526893

f1_score: 0.024, fp: 0.191, fn: 0.028
thr: 0.599514225012
roiScale: 7.20393280734
filtratio: 0.741440492092
black_h_lower: 16.4357595209
black_h_upper: 76.7782752282
black_s_lower: 111.561926049

f1_score: 0.023, fp: 0.18, fn: 0.031
thr: 0.599570119118
roiScale: 7.49614550505
filtratio: 0.749906770984
black_h_lower: 16.2565758116
black_h_upper: 77.9885521514
black_s_lower: 110.567314463

f1_score: 0.023, fp: 0.173, fn: 0.031
thr: 0.59982573146
roiScale: 7.25176854762
filtratio: 0.749900489885
black_h_lower: 17.4232033545
black_h_upper: 76.7956898821
black_s_lower: 110.416594497

f1_score: 0.023, fp: 0.196, fn: 0.022
thr: 0.599174127465
roiScale: 6.59198271318
filtratio: 0.741653027669
black_h_lower: 15.0453948092
black_h_upper: 76.7119827194
black_s_lower: 111.644292759

{'black_h_lower': 15.045394809193303,
 'black_h_upper': 76.71198271941329,
 'black_s_lower': 111.64429275915322,
 'filtratio': 0.741653027668764,
 'roiScale': 6.591982713182058,
 'threshold': 9.983482549303714}